In [1]:
import sys, os, glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import Slider
import shutil
sys.path.append('../..')
from src import raw_data_path, direcslist, load_image, def_roi, ImageQuant, ImageQuant2
%matplotlib widget

### Initialise ROI

In [2]:
# path = raw_data_path + '/Imaging/Meiosis'
# paths = direcslist(path, 1)
# imgs = [load_image(raw_data_path + glob.glob(p + '/*_Meiosis_*.tif')[0])[:, :, :, 0] for p in paths]
# for img, p in zip(imgs, paths):
#     roi = def_roi(img, periodic=True, spline=True)
#     np.savetxt(p + '/ROI.txt', roi)

### Select timepoints (cytokinesis)

In [3]:
# path = [raw_data_path + '/Imaging/meiosis/LP637', raw_data_path + '/Imaging/meiosis/NWG376']
# paths = direcslist(path, 0)
# imgs = [load_image(raw_data_path + glob.glob(p + '/*_Meiosis_*.tif')[0])[:, :, :, 0] for p in paths]

# def select_timepoint(image):
#     plt.ion()
#     fig, ax = plt.subplots()
#     plt.subplots_adjust(left=0.25, bottom=0.25)
#     axframe = plt.axes([0.25, 0.1, 0.65, 0.03])
#     sframe = Slider(axframe, 'Frame', -1, len(image), valinit=0, valfmt='%d')

#     def update(i):
#         ax.clear()
#         ax.imshow(image[int(i)], cmap='gray')
#         ax.set_xticks([])
#         ax.set_yticks([])

#     def key_press_callback(event):
#         if event.key == 'enter':
#             plt.close()

#     sframe.on_changed(update)
#     update(0)
#     fig.canvas.mpl_connect('key_press_event', key_press_callback)
#     plt.show(block=True)
#     return int(sframe.val)

# for img, p in zip(imgs, paths):
#     print(p)
#     timepoint = select_timepoint(img)
#     np.savetxt(p + '/cytokinesis.txt', [timepoint + 1], fmt='%i')

### Extract data from paths

In [4]:
path = [raw_data_path + '/Imaging/meiosis/LP637', raw_data_path + '/Imaging/meiosis/NWG376']
paths = direcslist(path, 0)
lines = []
embryo_ids = []
cytokinesis = []
for p in paths:
    lines.append(p.split('/')[-3])
    cytokinesis.append(int(np.loadtxt(p + '/cytokinesis.txt')))
    embryo_ids.append(int(str(os.path.getctime(glob.glob(p + '/*_Meiosis_*')[0])).replace('.', '')))
df = pd.DataFrame({'Path': paths, 'EmbryoID': embryo_ids, 'Line': lines, 'Cytokinesis': cytokinesis})
df['Path'] = df['Path'].apply(lambda x: x[len(raw_data_path):])
df.to_csv('../../data/meiosis_embryos.csv')
df

,Path,EmbryoID,Line,Cytokinesis
0,/Imaging/meiosis/LP637/20230529_Meiosis_LP637_...,16859571848951633,LP637,32
1,/Imaging/meiosis/LP637/20230531_Meiosis_LP687_e1/,16859571848608906,LP637,23
2,/Imaging/meiosis/LP637/20230531_Meiosis_LP687_e2/,16859571857754734,LP637,26
3,/Imaging/meiosis/LP637/20230531_Meiosis_LP687_e3/,16859571861310163,LP637,26
4,/Imaging/meiosis/NWG376/20230529_Meiosis_NWG37...,16859571829121623,NWG376,36
5,/Imaging/meiosis/NWG376/20230529_Meiosis_NWG37...,1685957181630857,NWG376,34
6,/Imaging/meiosis/NWG376/20230531_Meiosis_NWG37...,16859571835820549,NWG376,31
7,/Imaging/meiosis/NWG376/20230531_Meiosis_NWG37...,16859571840539403,NWG376,33
8,/Imaging/meiosis/NWG376/20230531_Meiosis_NWG37...,16859571844666913,NWG376,31


### Segmentation

In [5]:
# for index, row in df.iterrows(): 
    
#     # Import movie
#     img = load_image(raw_data_path + glob.glob(row.Path + '/*_corrected_*.tif')[0])[:row.Cytokinesis]
#     roi = np.loadtxt(raw_data_path + row.Path + '/ROI.txt')
    
#     # Set up optimiser
#     iq = ImageQuant(img=img, roi=roi, sigma=3.5, descent_steps=300, rol_ave=1, 
#                     lr=0.01, iterations=1, fit_outer=True, roi_knots=20, freedom=35, thickness=70)

#     # Segmentation program
#     iq.run()
#     iq.iq.adjust_roi()
#     iq.roi = iq.iq.roi
#     iq.iq.roi_knots = 30
#     iq.iq.thickness = 50
#     iq.iq.freedom = 5
#     iq.run()
#     iq.iq.adjust_roi()
#     iq.roi = iq.iq.roi
    
#     # Save
#     if os.path.exists(row.Path + '/Segmentation'):
#         shutil.rmtree(row.Path + '/Segmentation')
#     os.mkdir(row.Path + '/Segmentation')
#     for e, r in enumerate(iq.roi):
#         np.savetxt(row.Path + '/Segmentation/ROI_fit_%s.txt' % (e + 1), r)

### Monitor segmentation

In [6]:
# for index, row in df.iterrows(): 
#     img = load_image(raw_data_path + glob.glob(row.Path + '/*_corrected_*.tif')[0])[:row.Cytokinesis]
#     roi = [np.loadtxt(raw_data_path + row.Path + '/Segmentation/ROI_fit_%s.txt' % (i + 1))  for i in range(len(img))]
#     iq = ImageQuant(img, roi)
#     iq.plot_segmentation()
#     plt.show()

### Load quantification model

In [7]:
membg = np.loadtxt('../../data/calibration_mem6.txt')
cytbg = np.loadtxt('../../data/calibration_cyt.txt')
calibration_factor = np.loadtxt('../../data/calibration_constants.txt')[6]
print(calibration_factor)

3.128136976235888


### Quantification

In [8]:
df = pd.read_csv('../../data/meiosis_embryos.csv')
df['Path'] = df['Path'].apply(lambda x: raw_data_path + x)
results = []
for index, row in df.iterrows():
    print(row.Path)
    
    # Import movie
    img = load_image(glob.glob(row.Path + '/*_corrected_*.tif')[0])[:row.Cytokinesis]
    roi = [np.loadtxt(row.Path + '/Segmentation/ROI_fit_%s.txt' % (i + 1))  for i in range(len(img))]
    
    # Run quantification
    iq = ImageQuant2(img=img, roi=roi, cytbg=cytbg, membg=membg)
    iq.quantify(descent_steps=1000)
    # iq.plot_losses(log=True)
    
    # Compile results
    res = iq.compile_res(ids=row.EmbryoID * np.ones(iq.nimgs), extra_columns={'Frame': np.arange(iq.nimgs)})
    res['Membrane signal'] *= calibration_factor
    res = res.astype({'Frame': int})
    results.append(res)

# Compile all quantification data
results_full = pd.concat(results)

# Save
results_full.to_csv('../../data/meiosis_quantification.csv')

/Users/blandt/Desktop/PaperData//Imaging/meiosis/LP637/20230529_Meiosis_LP637_60x_e1/


  0%|                                                                                                                     | 0/1000 [00:00<?, ?it/s]

  0%|                                                                                                             | 1/1000 [00:00<11:39,  1.43it/s]

  1%|▉                                                                                                            | 9/1000 [00:00<01:07, 14.64it/s]

  2%|█▋                                                                                                          | 16/1000 [00:00<00:39, 25.18it/s]

...

 98%|████████████████████████████████████████████████████████████████████████████████████████████████████████▌  | 977/1000 [00:16<00:00, 61.32it/s]

 98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████▎ | 984/1000 [00:16<00:00, 61.06it/s]

 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████ | 991/1000 [00:16<00:00, 61.96it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 998/1000 [00:16<00:00, 61.42it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:17<00:00, 58.77it/s]